# 1. Library Import

In [1]:
# system path config
# import os
# import sys
# sys.path.append('/nfs/users/yihao/datalake-vis/src')   # make this the src folder in project home
# from datalake_vis.vis_instance.opt_vis_instance import OptPerfVisInstance
# from datalake_vis.utils import plot_vis_plan

# general import
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict

# 2. Parameter Sepcification

In [3]:
# Specify some necessary parameters

tus_res_path = "data/santos/tus_results/results_k50_t6.pkl"         # table union search result path, please specify a pickle file
# TBD: list of all required query table names for user study
query_table_name = "animal_tag_data_b.csv"           # the file name of the query table
query_path = "data/santos/query"           # path to the directory that contains queries
datalake_path = "data/santos/datalake"        # path to the directory that contains datalake
n = 10                     # the number of result tables


# load table union search
tus_res = pickle.load(open(tus_res_path, "rb"))
result_table_names = [x[0] for i, x in enumerate(tus_res[query_table_name]) if i < n]
result_table_paths = [f"{datalake_path}/{x[0]}" for i, x in enumerate(tus_res[query_table_name]) if i < n]
# orig_result_table_paths = [f"{orig_datalake_path}/{x[0]}" for i, x in enumerate(tus_res[query_table_name])]

print(result_table_names)
                                                                      


['animal_tag_data_b.csv', 'animal_tag_data_3.csv', 'animal_tag_data_1.csv', 'animal_tag_data_0.csv', 'animal_tag_data_6.csv', 'animal_tag_data_9.csv', 'animal_tag_data_8.csv', 'animal_tag_data_4.csv', 'animal_tag_data_7.csv', 'animal_tag_data_a.csv']


# 3. Data Loading

In [5]:
def load_csv_files(file_list: List[str]) -> List[Tuple[str, pd.DataFrame]]:
    dataframes = []
    for file in file_list:
        df = pd.read_csv(file)
        dataframes.append(df)
    return dataframes


table_dfs = load_csv_files([f"{query_path}/{query_table_name}"] + result_table_paths)



In [ ]:
print(len(table_dfs))
cnt = 0
for df in table_dfs:
    if 'price' in df:
        grp = df.groupby('animal_type').agg({'price':['sum']})
        df.groupby(by=["animal_type", "primary_color"]).size()['CAT']
        print(grp['price']['sum']['DOG'])
        print("==========")
# print(cnt)
# print(table_dfs[0]['DEPT_EN_DESC'].value_counts())
#print(sum(table_dfs[0]['AGRG_PYMT_AMT'])/ len('AGRG_PYMT_AMT'))

## 4. Data Summary

In [7]:
def print_schemas(df_list: List[Tuple[str, pd.DataFrame]]):
    for i, (df, name) in enumerate(zip(df_list, [query_table_name] + result_table_names)):
        print(i, name, df.columns)

print_schemas(table_dfs)

# Feel free to print further information if needed


0 animal_tag_data_b.csv Index(['animal_id', 'animal_type', 'breed_group', 'primary_breed', 'tag_no',
       'tag_type', 'tag_subtype', 'price', 'tag_date', 'tag_exp', 'tag_stat'],
      dtype='object')
1 animal_tag_data_b.csv Index(['animal_id', 'animal_type', 'breed_group', 'primary_breed', 'tag_no',
       'tag_type', 'tag_subtype', 'price', 'tag_date', 'tag_exp', 'tag_stat'],
      dtype='object')
2 animal_tag_data_3.csv Index(['animal_id', 'animal_type', 'breed_group', 'primary_breed', 'tag_no',
       'tag_type', 'tag_subtype', 'price', 'tag_date', 'tag_exp', 'tag_stat'],
      dtype='object')
3 animal_tag_data_1.csv Index(['animal_id', 'animal_type', 'breed_group', 'primary_breed', 'tag_no',
       'tag_type', 'tag_subtype', 'price', 'tag_date', 'tag_exp', 'tag_stat'],
      dtype='object')
4 animal_tag_data_0.csv Index(['animal_id', 'animal_type', 'breed_group', 'primary_breed', 'tag_no',
       'tag_type', 'tag_subtype', 'price', 'tag_date', 'tag_exp', 'tag_stat'],
      dtype=

In [9]:
# Answer to Q1
ans = set()
for df in table_dfs:
    if 'animal_type' in df:
        s = dict(df['animal_type'].value_counts()) 
        for k,_ in s.items():
            ans.add(k)
print(ans)

{'NORMAL', 'GUINEA PIG', 'FERRET', 'RABBIT', 'CAT', 'BIRD', 'DOG', 'OTHER', 'LIVESTOCK'}


In [11]:
# Answer to Q2
group_cnt = dict()
ans = []
for df in table_dfs:
    if 'breed_group' in df:
        s = dict(df['breed_group'].value_counts())
        for k,v in s.items():
            if k in group_cnt:
                group_cnt[k] += v
            else:
                group_cnt[k] = v
for k,v in group_cnt.items():
    if v > 1000:
        ans.append(k)
print(ans)

['SHORTHAIR', 'TOY', 'MASTIFF', 'SETTER/RETRIEVE', 'TERRIER', 'HOUND', 'SHEPHERD', 'COLLIE', 'LONGHAIR', 'POODLE', 'SPITZ', 'MIX', 'POINTER', 'SPANIEL']


In [13]:
# Answer to Q3
colors = dict()
for df in table_dfs:
     if 'primary_color' in df and 'animal_type' in df:
         grp = dict(df.groupby(by=["animal_type", "primary_color"]).size()['CAT'])
         for k,v in grp.items():
             if k in colors:
                colors[k] += v
             else:
                colors[k] = v
colors_rank = [(k,v) for k, v in sorted(colors.items(), key=lambda item: item[1])]
ans = colors_rank[-2]
print(ans)

('GRAY', 563)


In [15]:
# Answer to Q4
total_price = 0.0
total_cnt = 0.0
for df in table_dfs:
    if 'price' in df and 'animal_type' in df:
        grp = df.groupby('animal_type').agg({'price':['sum']})
        total_price += grp['price']['sum']['RABBIT']
        total_cnt += df.groupby(by=["animal_type"]).size()['RABBIT']
ans = total_price / total_cnt
print(ans)

7.777777777777778


# 5. Table Pre-processing and Selection

In [ ]:
# enter the index of table you would like to explore
selected_1 = [0, 1, 2, 3, 4, 8]
table_dfs_1 = [table_dfs[i] for i in selected_1]
result_table_names_1 = [result_table_names[i - 1] for i in selected_1 if i > 0]

selected_2 = [5, 6, 7, 8, 9, 10]
table_dfs_2 = [table_dfs[i] for i in selected_2]
result_table_names_2 = [result_table_names[i - 1] for i in selected_2 if i > 0]

# add any preprocessing for the table here



# 6. Perform Group-By and Calculate Aggregate

In [ ]:
def prepare_dataframe_for_grouping(table_dfs: List[pd.DataFrame], matching_info: Tuple[str, List[Tuple[int, int]]], x_column: str, y_column: str) -> pd.DataFrame:
    x_col_idx, y_col_idx = table_dfs[0].columns.get_loc(x_column), table_dfs[0].columns.get_loc(y_column)
    matching = {t[0]: {p[0]: p[1] for p in t[1]} for t in matching_info}
    x_df = pd.concat([table_dfs[0][x_column]] + [table_dfs[i].iloc[:,matching[result_table_names[i-1]][x_col_idx]] for i, t in enumerate(table_dfs) if i > 0], axis=0)
    y_df = pd.concat([table_dfs[0][y_column]] + [table_dfs[i].iloc[:,matching[result_table_names[i-1]][y_col_idx]] for i, t in enumerate(table_dfs) if i > 0], axis=0)
    return pd.concat([x_df, y_df], axis=1)

def perform_group_by(df: pd.DataFrame, x_column: str, y_column: str, agg_func: str) -> pd.DataFrame:
    result = df.sort_values(x_column).groupby(x_column).agg({y_column: agg_func})
    return result
    
# specify the columns in the query table
x_column = "animal_type"
y_column = "price"
agg_func = "mean"
group_df_1 = prepare_dataframe_for_grouping(table_dfs_1, tus_res[query_table_name], x_column, y_column)
grouped_result_1 = perform_group_by(group_df_1, x_column, y_column, agg_func)

#group_df_2 = prepare_dataframe_for_grouping(table_dfs_2, tus_res[query_table_name], x_column, y_column)
#grouped_result_2 = perform_group_by(group_df_2, x_column, y_column, agg_func)


# 7. Lib functions for users

In [ ]:
# TODO: add more descriptions about input/output
def print_schemas(df_list):
    # copy from above
    pass

def prepare_dataframe_for_grouping(table_dfs, matching_info, x_column, y_column):
    # copy from above
    pass

def perform_group_by(df, x_column, y_column, agg_func):
    # copy from above
    pass

def select_subset_of_tables(table_dfs, result_table_names, list_idx):
    selected_table_dfs = [table_dfs[i] for i in list_idx]
    selected_table_names = [result_table_names[i - 1] for i in list_idx if i > 0]
    return selected_table_dfs, selected_table_names

def column_projection(table_dfs, idx, col_name):
    return table_dfs[idx][col_name]

def count_all_cate_value(df, col):
    return df[col].value_counts()


In [ ]:
# example question: which fiscal year (FSCL_YR) has the second most number of records? 
for df in table_dfs:
    print(count_all_cate_value(df,'FSCL_YR'))
# Answer: 2011/2012 

# (Removed) Visualization of the Aggregate Result

In [ ]:
# Function to visualize the group-by result
def visualize_group_by_result(df: pd.DataFrame, x_column: str, y_column: str, chart_type: str = "bar"):
    if chart_type == "bar":
        df.plot(kind='bar', x=x_column, y=y_column)
    elif chart_type == "line":
        df.plot(kind='line', x=x_column, y=y_column)
    elif chart_type == "scatter":
        df.plot(kind='scatter', x=x_column, y=y_column)
    else:
        raise ValueError("Unsupported chart type")
    
    plt.xlabel(x_column)
    plt.ylabel(y_column)
    plt.title(f"{chart_type.capitalize()} chart of {y_column} grouped by {x_column}")
    plt.show()

# Example usage
chart_type = "bar"  # Replace with desired chart type ('bar', 'line', 'scatter', etc.)
visualize_group_by_result(grouped_result_1.reset_index(), x_column, y_column, chart_type)

In [ ]:
visualize_group_by_result(grouped_result_2.reset_index(), x_column, y_column, chart_type)
# plot_vis_plan(top_k_plans[0])
# print(top_k_plans[0].plan_details())